<a href="https://colab.research.google.com/github/stefymojica/MACC/blob/main/Modelo_clinico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Agente para la optimizacion de recursos en un modelo clinico
## Matematicas aplicadas y ciencias de la computacion
## Estudiante : Stefany Paola Mojica Melo

## Contextualizacion

- Hay 3 agentes:
  - pacientes
  - medicos
  - triaja
- Hay un modelo
  - Clinica donde los agentes van a interactuar
- Los pacientes llegan y el triaje asigna el nivel de riesgo dependiendo de la situacion del paciente
- A la clinica llegan cada 25 minutos un nuevo paciente
- Los doctores se demoran 15 minutos para descubrir que le sucede a un paciennte -
- los pacientes llegan y el triaje les asigana un nivel de criticidad son 5 niveles.
  - nivel 1 - situacion de riesgo vital, requiere accion inmediata
  - nivel 2 - Emergencia. requiere atencion en un tiempo maximo de 30 minutos
  - nivel 3 - Urgencia. se debe atender en un tiempo no mayor a 30 minutos
  - nivel 4 - urgencia menor. Puede esperar hasta 120 minutos
  - nivel 5 - sin urgencia. puede esperar hasta 180 minutos

## Instalar librerias

In [1]:
%pip install mesa

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 kB 5.4 MB/s eta 0:00:00


In [2]:
import mesa

In [21]:
import mesa
print("Versión de Mesa instalada:", mesa.__version__)

Versión de Mesa instalada: 3.3.0


In [23]:
!pip uninstall -y mesa

Found existing installation: Mesa 3.3.0
Uninstalling Mesa-3.3.0:
  Successfully uninstalled Mesa-3.3.0


In [20]:
from mesa import Agent, Model
from mesa.space import MultiGrid
from mesa.time import RandomActivation
from mesa.datacollection import DataCollector
import random
import heapq

ModuleNotFoundError: No module named 'mesa.time'

## Creacion de Agentes y Modelo

In [18]:
class Paciente(Agent):
  def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.prioridad = None
        self.atendido = False

  def step(self):
    pass

class Triaje(Agent):
  def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.ocupado = False

  def step(self):
    if not self.ocupado and len(self.model.pacientes_sin_triaje) > 0:
      paciente = self.model.pacientes_sin_triaje.pop(0)
      self.asignar_prioridad(paciente)

  def asignar_prioridad(self, paciente):
    r = random.random()
    if r < 0.05: paciente.prioridad = 1
    elif r < 0.20: paciente.prioridad = 2
    elif r < 0.50: paciente.prioridad = 3
    elif r < 0.80: paciente.prioridad = 4
    else: paciente.prioridad = 5

    heapq.heappush(self.model.cola_prioridad_triaje, (paciente.prioridad, paciente))

class Doctor(Agent):
  def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.ocupado = False
        self.paciente_actual = None

  def step(self):
    if not self.ocupado and len(self.model.cola_prioridad) > 0:
      _, paciente = heapq.heappop(self.model.cola_prioridad_triaje)
      self.atender(paciente)

    elif self.ocupado:
      self.ocupado = False
      self.paciente_actual = None

  def atender(self, paciente):
    self.ocupado = True
    self.paciente_actual = paciente
    paciente.atendido = True
    self.model.pacientes_atendidos += 1

class Clinica(Model):
  def __init__(self, n_doctores, tasa_llegada):
    super().__init__()
    #self.schedule = mesa.time.RandomActivation(self)
    self.n_doctores = n_doctores
    self.tasa_llegada = tasa_llegada
    self.pacientes_sin_triaje = []
    self.cola_prioridad_triaje = []
    self.pacientes_atendidos = 0
    self.tick = 0


    for d in range(n_doctores):
      doctor = Doctor(d, self)
      print(doctor)
      #self.schedule.add(doctor)

    self.triaje = Triaje("T1", self)
    #self.schedule.add(self.triaje)

  def step(self):
    if random.random() < self.tasa_llegada:
      paciente = Paciente(f"Paciente{self.tick}", self)
      #self.schedule.add(paciente)
      self.pacientes_sin_triaje.append(paciente)

    self.triaje.step()
    for doctor in self.schedule.agents:
      doctor.step()
    self.tick += 1

In [19]:
model = Clinica(n_doctores=2, tasa_llegada=0.7)
for i in range(20):
  model.step()
  #print(f"Tick: {i} | Pacientes atendidos: {model.pacientes_atendidos} | Pacientes en Cola:{len(model.cola_prioridad_triaje)}")

TypeError: object.__init__() takes exactly one argument (the instance to initialize)